In [1]:
import os
import math
import time
import numpy as np
import multiprocessing as mp
from collections import Counter
from os.path import getsize

os.chdir("/Users/kobinae/KOBITemp/Dropbox/10_Venturus/MDC/201810_INF-0617/Ex01/INF0617_Ex01/gut/txt")
#os.chdir("/home/venturus/Dropbox/10_Venturus/MDC/201810_INF-0617/Ex01")

filenames = []
for file in os.listdir("./"):
    if file.endswith(".txt"):
        filenames.append(file)
filenames_sorted = sorted(filenames, key=getsize)

def mysplit(filenames,N,verbose=False):
    #filenames = sorted(filenames, key=getsize)
    #SPLITING A LIST OF FILES TO N CHUNKS WITH SIMILAR TOTAL SIZE IN MB
    s=[0]*N #filesize for each chunck
    ms = [[] for _ in range(N)]
    for f in filenames:
        n=s.index(min(s))
        ms[n].append(f)
        s[n]=s[n]+os.path.getsize(f)
    if verbose:
        for i in range(N):
            print("chunck",i,"nfiles=",len(ms[i])," size=",s[i])
    return ms

def proc_single_file(filename,q):
    #FINDING THE MOST REPEATED CHAR IN A SPECIFIC FILE
    f = open(filename, encoding='utf-8', errors='ignore')
    file_content = f.read()
    f.close()
    new_content = file_content.replace(' ', '').replace('\n','')
    q.put(filename + ' ' + Counter(new_content).most_common(1)[0][0])

def proc_array_file(array_file,q):
    #SERIAL IMPLEMENTATION FOR FINDING MOST REPEATED CHAR IN LIST OF FILES
    for f in array_file:
        proc_single_file(f,q)
    return q
        
def count_char_parallel(filenames,nsplit,verbose):
    #PARALLEL IMPLEMENTATION FOR FINDING MOST REPEATED CHAR IN LIST OF FILES
    file_sets=mysplit(filenames,nsplit,verbose)
    q = mp.Queue()
    processes = []
    for f_set in file_sets:
        p = mp.Process(target=proc_array_file, args=(f_set,q))
        p.start()
        processes.append(p)
    for p in processes:
        p.join()
    if verbose:
        while not q.empty():
            print(q.get())
    return q

In [2]:
#EXECUTION
tmp_time = time.time()
q = mp.Queue()
q_serial=proc_array_file(filenames,q)
print('Serial execution took ' + str(time.time()-tmp_time) + ' seconds')

tmp_time = time.time()
q_parallel=count_char_parallel(filenames,4,False)
print('Parallel execution took ' + str(time.time()-tmp_time) + ' seconds')


Serial execution took 27.95413088798523 seconds
Parallel execution took 13.105098009109497 seconds


In [3]:
#VALIDATION
a=[]
while not q_parallel.empty():
    a.append(q_parallel.get())
b=[]
while not q_serial.empty():
    a.append(q_serial.get())
if a.sort() == b.sort():
    print("Parallel and serial results are the same")

Parallel and serial results are the same
